In [1]:
import pandas as pd
import numpy as np
import re

import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

ModuleNotFoundError: No module named 'tensorflow_hub'

In [2]:
dev_df=pd.read_csv("civility_data/dev.tsv",sep="\t",header=0)
mini_demodev_df=pd.read_csv("civility_data/mini_demographic_dev.tsv",sep="\t",header=0)

In [3]:
mini_demodev_df=mini_demodev_df.fillna(0)

In [4]:
dev_df.loc[dev_df.perspective_score>0.8,"label_ps"]='OFF'
dev_df.loc[dev_df.perspective_score<=0.8,"label_ps"]='NOT'

In [5]:
mini_demodev_df.loc[mini_demodev_df.perspective_score>0.8,"label_ps"]='OFF'
mini_demodev_df.loc[mini_demodev_df.perspective_score<=0.8,"label_ps"]="NOT"

In [17]:
class Matrics:
    def __init__(self,targets,gold):
        self.tp=0
        self.tn=0
        self.fp=0
        self.fn=0
        self.targetsize=len(targets)
        for i in range(len(targets)):
            if targets[i] ==1. and gold[i] ==1.: # classify offensive claim as OFF
                self.tp+=1
            elif targets[i] == 0. and gold[i] == 0.: # classify non-offensive claim as NOT
                self.tn+=1
            elif targets[i] == 0. and gold[i] ==1.: # classify offensive claim as NOT
                self.fp+=1
            elif targets[i] == 1. and gold[i]== 0.: # classify non-offensive claim as OFF
                self.fn+=1

    def f1_score(self):
        p=self.precision()
        r=self.recall()
        return 2*((p*r)/(p+r))

    def precision(self):
        tp=self.tp
        fp=self.fp
        return tp/(tp+fp)

    def recall(self):
        tp = self.tp
        fn = self.fn
        return tp/(tp+fn)

    def accuracy(self,targets,gold):
        dif=0
        for x,y in zip(targets,gold):
            if x!=y:
                dif+=1
        return dif/len(targets)

    def PFR_score(self):
        return self.fn/self.targetsize

    def summary(self,targets,golds):
        print("The accuracy is {0}".format(self.accuracy(targets,golds)))
        print("The F1_score is {0}".format(self.f1_score()))

def normalize_label(labels):
    '''
    encode label to 0,1. 'OFF' to 1, 'NOT' to 0
    '''
    labels_encoded=[]
    for label in labels:
        if label=='OFF':
            labels_encoded.append(1)
        elif label=='NOT':
            labels_encoded.append(0)
    if len(labels_encoded)!=len(labels):
        raise ValueError
    else:
        return labels_encoded


In [18]:
print('-'*30+'1.1.b Starting '+'-'*30)
dev_y=dev_df.label.tolist()
dev_X=dev_df.label_ps.tolist()
dev_y=normalize_label(dev_y)
dev_X=normalize_label(dev_X)
m=Matrics(dev_X,dev_y)
m.summary(dev_X,dev_y)
print('-'*30+'1.1.b Ending '+'-'*30)

------------------------------1.1.b Starting ------------------------------
The accuracy is 0.23564954682779457
The F1_score is 0.4851485148514852
------------------------------1.1.b Ending ------------------------------


In [20]:
print('-'*30+'1.1.c Starting '+'-'*30)
mini_demodev_X=mini_demodev_df.label_ps.tolist()
mini_demodev_X=normalize_label(mini_demodev_X)
mini_demodev_y=np.zeros(len(mini_demodev_X)).tolist()
m=Matrics(mini_demodev_X,mini_demodev_y)
print("The False Positive Rate is {0}".format(m.PFR_score()))
print('-'*30+'1.1.c Ending '+'-'*30)

------------------------------1.1.c Starting ------------------------------
The False Positive Rate is 0.08063880126182965
------------------------------1.1.c Ending ------------------------------


In [22]:
print('='*30+'1.2 Custom Classifier Starting '+'='*30)

==============================1.1.c Custom Classifier Starting ==============================


In [33]:
def read_data(filepath):
    return pd.read_csv(filepath,sep='\t',header=0)

def preprocessing(sentence):
    result=' '.join(re.sub("(@[A-Za-z0-9]+)"," ",sentence).split())
    print(result)

def word_embedding():





tmp='@USER @USER Who really listens to those post menapausal angry yentas except hardcore liberals and femanazis?'
preprocessing(tmp)

Who really listens to those post menapausal angry yentas except hardcore liberals and femanazis?


In [23]:

print('='*30+'1.2 Ending '+'='*30)

==============================1.2 Ending ==============================


In [ ]:
def inspect_NaN(df):
    is_nan=df.isna()
    has_nan_row=is_nan.any(axis=1)
    print(df[has_nan_row])